In [90]:
import json
import os
from sklearn.feature_extraction.text import TfidfVectorizer
import rank_bm25 as bm25
import pandas as pd
import numpy as np
import re
import string
from gensim.parsing.preprocessing import remove_stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import nltk
nltk.download('wordnet')
import xmltodict

[nltk_data] Downloading package wordnet to /home/mimig/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [91]:
# from rank_bm25 import BM25Okapi

In [92]:
wd = '/media/sf_year2/DD2430-Project/Podcast-Segment-Retrieval-Spotify/'
os.chdir(wd)
files = os.listdir('./data/training_episodes')
files = [f for f in files if f != 'place_holder']
# files

In [93]:
from utils_stats import *

In [94]:
metadata_file = "./data/metadata.tsv"
metadata = pd.read_csv(metadata_file, sep="\t")
metadata['show_id'] = metadata.apply(lambda x: x['show_uri'].split(":")[2], axis=1)
metadata['episode_id'] = metadata.apply(lambda x: x['episode_uri'].split(":")[2], axis=1)
metadata.set_index('episode_id', inplace=True)
# metadata.head()

In [95]:
qrels_file = "./data/podcasts_2020_train.1-8.qrels.txt"
qrels = pd.read_csv(qrels_file, sep="\t", header=None)#
qrels.set_axis(['topic', '1', 'episode_start', 'score'], axis=1, inplace=True)
qrels['episode'] = qrels.apply(lambda x: x['episode_start'].split(":")[2].split("_")[0], axis=1)
qrels['start'] = qrels.apply(lambda x: int(x['episode_start'].split("_")[1].split(".")[0]), axis=1)
qrels['end'] = qrels.apply(lambda x: int(x['start'] + 120), axis=1)
qrels['interval'] = qrels.apply(lambda x: (x['start'], x['end']), axis=1)
# qrels = qrels[qrels.score>0]
qrels

,topic,1,episode_start,score,episode,start,end,interval
0,1,0,spotify:episode:0E2nqCXMkS218SE72APmNr_240.0,2,0E2nqCXMkS218SE72APmNr,240,360,"(240, 360)"
1,1,0,spotify:episode:0E2nqCXMkS218SE72APmNr_300.0,2,0E2nqCXMkS218SE72APmNr,300,420,"(300, 420)"
2,1,0,spotify:episode:0E2nqCXMkS218SE72APmNr_360.0,2,0E2nqCXMkS218SE72APmNr,360,480,"(360, 480)"
3,1,0,spotify:episode:0Th494DvnO5dU8vTi3QHm2_120.0,1,0Th494DvnO5dU8vTi3QHm2,120,240,"(120, 240)"
4,1,0,spotify:episode:199bOiXL0l4YsRaSNDNXvP_1200.0,2,199bOiXL0l4YsRaSNDNXvP,1200,1320,"(1200, 1320)"
...,...,...,...,...,...,...,...,...
604,8,0,spotify:episode:7MnlmF9PV3WUOPI8IjGd0y_720.0,0,7MnlmF9PV3WUOPI8IjGd0y,720,840,"(720, 840)"
605,8,0,spotify:episode:7MnlmF9PV3WUOPI8IjGd0y_780.0,0,7MnlmF9PV3WUOPI8IjGd0y,780,900,"(780, 900)"
606,8,0,spotify:episode:7wJUUYWcHm7YwEoCmTGFoD_240.0,1,7wJUUYWcHm7YwEoCmTGFoD,240,360,"(240, 360)"
607,8,0,spotify:episode:7wJUUYWcHm7YwEoCmTGFoD_300.0,1,7wJUUYWcHm7YwEoCmTGFoD,300,420,"(300, 420)"


In [113]:
with open("./data/training_sub_episode_loc.out", "r") as f:
    files = f.readlines()
    
files = [f.replace("\n", "") for f in files]
# files

In [97]:
def is_overlapping(x, y):
    return x[0] <= y[1] and y[0] <= x[1]

In [98]:
# training_sub = {}

# for filename in files: 
#     training_sub[os.path.splitext(filename)[0]] = extract_segments(filename)

# with open("./data/training_sub_cleaned.json", "w") as f:
#     json.dump(training_sub, f)

with open("./data/training_sub_cleaned.json", "r") as f:
    training_sub = json.load(f)

In [99]:
tf_idf, ix2episode, full_texts = get_tf_idf(training_sub)
topics = extract_topics("./data/podcasts_2020_topics_train.xml")

In [100]:
# for ti, topic_dict in topics.items():
#     print("Topic {} {}: {}".format(ti, topic_dict['query'], topic_dict['description']))
# #     print("The topic is: {}".format(topic))
#     top_episodes = get_topic_episodes(topics, ti, tf_idf, ix2episode)
#     for ei, e in enumerate(top_episodes):
        
#         print("Result {}: \n{}: {}".format(ei, metadata.episode_name[e], metadata.episode_description[e]))
#     print("\n")
ti = 8
topic_dict = topics[ti]
print("Topic {} {}: {}".format(ti, topic_dict['query'], topic_dict['description']))
#     print("The topic is: {}".format(topic))
top_episodes = get_topic_episodes(topics, ti, tf_idf, ix2episode, n_episodes=60)
# for ei, e in enumerate(top_episodes):

#     print("Result {}: \n{}: {}".format(ei, metadata.episode_name[e], metadata.episode_description[e]))
# print("\n")

Topic 8 facebook stock prediction: After Facebook’s Q4 2018 earnings call, what were experts’ predictions and expectations for its stock price in 2019?  How did these predictions fare over time?  Relevant material would include predictions immediately following the Jan 29, 2019, earnings call, and later actual stock performance that could be used to evaluate predictions.


In [101]:
# flatten = lambda l: [item for sublist in l for item in sublist]
# flat_segments = flatten(all_segments)
# tokenized_segments = [x.split(" ") for x in flat_segments]
# tokenized_segments

In [102]:
i2episode = [t for t in top_episodes for sd in training_sub[t]['segments']]
i2segnum = [sd['segNum'] for t in top_episodes for sd in training_sub[t]['segments']]
i2transcript = [sd['transcript'] for t in top_episodes for sd in training_sub[t]['segments']]
i2transcript_clean = [sd['transcript_clean'] for t in top_episodes for sd in training_sub[t]['segments']]
i2tokens = [tokenize(x) for x in i2transcript_clean]
ix = list(range(len(i2segnum)))

print(len(i2episode), len(i2segnum), len(i2transcript), len(i2transcript_clean), len(i2tokens))

5382 5382 5382 5382 5382


In [103]:
# bm25 = BM25Okapi(tokenized_segments)

# query = "climate change strike"
# tokenized_query = query.split(" ")

# bm25.get_top_n(tokenized_query, flat_segments, n=2)

In [104]:
model = bm25.BM25Okapi(i2tokens)

tokenized_query = tokenize(topic_dict['query'])
tokenized_query.extend(tokenize(clean_text(topic_dict['description'])))
choices = model.get_top_n(tokenized_query, ix, 10)

In [105]:
def get_segment_details(choices, i2segnum, i2episode, i2transcript):
    result = {}
    for ix, i in enumerate(choices):
        result["spotify:episode:{}_{}.0".format(i2episode[i], 30*i2segnum[i])] = {
            "episode": i2episode[i],
            "start": 30*i2segnum[i],
            "end": 30*i2segnum[i] + 120,
            "interval": (30*i2segnum[i], 30*i2segnum[i] + 120),
            "transcript": i2transcript[i],
            "rank": ix
        }
        
    return result
        

In [106]:
top10 = get_segment_details(choices, i2segnum, i2episode, i2transcript)

In [107]:
list(top10.keys())

['spotify:episode:/media/sf_year2/DD2430-Project/Podcast-Segment-Retrieval-Spotify/data/training_episodes/5UGPiylWqD8IXIRflWXJEg_1620.0',
 'spotify:episode:/media/sf_year2/DD2430-Project/Podcast-Segment-Retrieval-Spotify/data/training_episodes/5UGPiylWqD8IXIRflWXJEg_1650.0',
 'spotify:episode:/media/sf_year2/DD2430-Project/Podcast-Segment-Retrieval-Spotify/data/training_episodes/55VfynKK09foFInjoBjldC_150.0',
 'spotify:episode:/media/sf_year2/DD2430-Project/Podcast-Segment-Retrieval-Spotify/data/training_episodes/0rBkvc7wrAf5sLXbMuk0QO_2760.0',
 'spotify:episode:/media/sf_year2/DD2430-Project/Podcast-Segment-Retrieval-Spotify/data/training_episodes/5EJg9dgAMD1HhlRSN99ttX_1350.0',
 'spotify:episode:/media/sf_year2/DD2430-Project/Podcast-Segment-Retrieval-Spotify/data/training_episodes/5UGPiylWqD8IXIRflWXJEg_780.0',
 'spotify:episode:/media/sf_year2/DD2430-Project/Podcast-Segment-Retrieval-Spotify/data/training_episodes/0GKJT6gpps3nvLZpJ6yvou_4410.0',
 'spotify:episode:/media/sf_year2/DD

In [108]:
topics

{1: {'query': 'coronavirus spread',
  'type': 'topical',
  'description': 'What were people saying about the spread of the novel coronavirus NCOV-19 in Wuhan at the end of 2019?',
  'description_clean': 'people saying spread novel coronavirus ncov wuhan end '},
 2: {'query': 'greta thunberg cross atlantic',
  'type': 'topical',
  'description': 'What were people saying about Greta Thunberg’s sailing trip across the Atlantic Ocean in the fall of 2019 and its relationship to global climate change?',
  'description_clean': 'people saying greta thunberg’s sailing trip atlantic ocean fall  relationship global climate change'},
 3: {'query': 'black hole image',
  'type': 'topical',
  'description': 'In May 2019 astronomers released the first-ever picture of a black hole.  I would like to hear some conversations and educational discussion about the science of astronomy, black holes, and of the picture itself.',
  'description_clean': ' astronomer released firstever picture black hole like hea

In [109]:
def compare_with_qrels(results, qrels, topic):
    df = qrels[qrels.topic == topic]
    for k, v in results.items():
        e = v['episode']
        i = v['interval']
#         print(np.sum(df.episode==e))
        if np.sum(df.episode==e)>0:
            ixs = df.episode == e
            for episode, interval, score in zip(df.episode_start[ixs], df.interval[ixs], df.score[ixs]):
                if is_overlapping(i, interval):
                    print("{} is overlapping with choice in qrels with score {}".format(k, score))

In [110]:
compare_with_qrels(top10, qrels, ti)

In [111]:
for ti, topic_dict in topics.items():

    print("Topic {} {}: {}".format(ti, topic_dict['query'], topic_dict['description']))
    top_episodes = get_topic_episodes(topics, ti, tf_idf, ix2episode)
    i2episode = [t for t in top_episodes for sd in training_sub[t]['segments']]
    i2segnum = [sd['segNum'] for t in top_episodes for sd in training_sub[t]['segments']]
    i2transcript = [sd['transcript'] for t in top_episodes for sd in training_sub[t]['segments']]
    i2transcript_clean = [sd['transcript_clean'] for t in top_episodes for sd in training_sub[t]['segments']]
    i2tokens = [tokenize(x) for x in i2transcript_clean]
    ix = list(range(len(i2segnum)))
    
    model = bm25.BM25Okapi(i2tokens)

    tokenized_query = tokenize(topic_dict['query'])
    tokenized_query.extend(tokenize(clean_text(topic_dict['description'])))
    choices = model.get_top_n(tokenized_query, ix, 10)
    
    top10 = get_segment_details(choices, i2segnum, i2episode, i2transcript)
    
    compare_with_qrels(top10, qrels, ti)
    print(" ")

Topic 1 coronavirus spread: What were people saying about the spread of the novel coronavirus NCOV-19 in Wuhan at the end of 2019?
 
Topic 2 greta thunberg cross atlantic: What were people saying about Greta Thunberg’s sailing trip across the Atlantic Ocean in the fall of 2019 and its relationship to global climate change?
 
Topic 3 black hole image: In May 2019 astronomers released the first-ever picture of a black hole.  I would like to hear some conversations and educational discussion about the science of astronomy, black holes, and of the picture itself.
 
Topic 4 story about riding a bird: I remember hearing a podcast that had a story about a kid riding some kind of bird.  I want to find it again.
 
Topic 5 daniel ek interview: Someone told me about a podcast interview with Daniel Ek, CEO of Spotify, about the founding and early days of Spotify.  I would like to find the show and episode that contains that interview.  Other interviews with Ek are relevant as well.
 
Topic 6 miche

In [112]:
for ti, topic_dict in topics.items():

    print("Topic {} {}: {}".format(ti, topic_dict['query'], topic_dict['description']))
    top_episodes = get_topic_episodes(topics, ti, tf_idf, ix2episode)
    i2episode = [t for t in top_episodes for sd in training_sub[t]['segments']]
    i2segnum = [sd['segNum'] for t in top_episodes for sd in training_sub[t]['segments']]
    i2transcript = [sd['transcript'] for t in top_episodes for sd in training_sub[t]['segments']]
    i2transcript_clean = [sd['transcript_clean'] for t in top_episodes for sd in training_sub[t]['segments']]
    i2tokens = [tokenize(x) for x in i2transcript_clean]
    ix = list(range(len(i2segnum)))
    
    model = bm25.BM25Okapi(i2tokens)

    tokenized_query = tokenize(topic_dict['query'])
    tokenized_query.extend(tokenize(clean_text(topic_dict['description'])))
    choices = model.get_top_n(tokenized_query, ix, 10)
    
    top10 = get_segment_details(choices, i2segnum, i2episode, i2transcript)
    
    for k, res in top10.items(): 
        print("{}: {}".format(res['rank'], res['transcript']))
    print(" ")

Topic 1 coronavirus spread: What were people saying about the spread of the novel coronavirus NCOV-19 in Wuhan at the end of 2019?
0: It started.  Hey, welcome back. So I want to to make today's episode about this continuing spread of the Wuhan coronavirus. I think the official name for it is the 2019 novel coronavirus, but many people are referring to it as just the coronavirus the the Wuhan coronavirus Wuhan flew I've seen that go around but and I think a good place to start is just a quick recap. Where were at now of course is subject to change.
1:  Hey, welcome back. So I want to to make today's episode about this continuing spread of the Wuhan coronavirus. I think the official name for it is the 2019 novel coronavirus, but many people are referring to it as just the coronavirus the the Wuhan coronavirus Wuhan flew I've seen that go around but and I think a good place to start is just a quick recap. Where were at now of course is subject to change.  And I'll be talking more about t